In [39]:
from collections import Counter 

import pandas as pd
import numpy as np
import re

In [40]:
data_path = "data/"
df = pd.read_csv(data_path + "cleaned_train.csv")

In [41]:
# df = df[:200]
df

,raw_address,POI/street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung
2,"toko dita , kertosono",toko dita/
3,jl. orde baru,/jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede
...,...,...
265883,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya
265884,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
265885,"raya cila kko , cilandak timur kel.",/raya cila kko
265886,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [42]:
def getPOI(s) :
    return s.split("/")[0]

def getStreet(s) :
    return s.split("/")[1]

df["POI"] = df["POI/street"].apply(getPOI)
df["Street"] = df["POI/street"].apply(getStreet) 

In [43]:
def create_label(s, poi, street) :
    tokens = s.split(" ")
    n = len(tokens)
    tp = poi.split(" ")
    ts = street.split(" ")
    tags = []
    i = 0
    while i < n :
        if len(tp) > 0 and tokens[i] == tp[0] :
            j = 1
            while j < len(tp) :
                if i+j < n and tokens[i+j] == tp[j]:
                    j += 1
                else :
                    break
            if j == len(tp) :
                tags.append("B-POI")
                for k in range(1, j) :
                    tags.append("I-POI")
                i += j
            else :
                tags.append("O")
                i += 1
        elif len(ts) > 0 and tokens[i] == ts[0] :
            j = 1
            while j < len(ts) :
                if i+j < n and tokens[i+j] == ts[j] :
                    j += 1
                else :
                    break
            if j == len(ts) :
                tags.append("B-STREET")
                for k in range(1, j) :
                    tags.append("I-STREET")
                i += j
            else :
                tags.append("O")
                i += 1                
        else :
            tags.append("O")
            i += 1
        
#         print(tokens)
#         print(tags)

    if tokens[-1] != "." :
        tokens.append(".")
        tags.append("O")
    
    res = ""
#     print(tokens)
#     print(tags)
    for token, tag in zip(tokens, tags) :
        res += "{}\t{}\n".format(token, tag)
    return res

In [44]:
s = "raya samb gede , 299 toko bb kids"
poi = "toko bb kids"
street = "raya samb gede"
res = create_label(s, poi, street)
print(res)

raya	B-STREET
samb	I-STREET
gede	I-STREET
,	O
299	O
toko	B-POI
bb	I-POI
kids	I-POI
.	O



In [45]:
df["Label"] = df.apply(lambda x: create_label(x.raw_address, x.POI, x.Street), axis=1)

In [46]:
df

,raw_address,POI/street,POI,Street,Label
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika,jl\tB-STREET\nkapuk\tI-STREET\ntimur\tI-STREET...
1,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung,setu\tO\nsiung\tB-STREET\n119\tO\nrt\tO\n5\tO\...
2,"toko dita , kertosono",toko dita/,toko dita,,"toko\tB-POI\ndita\tI-POI\n,\tO\nkertosono\tO\n..."
3,jl. orde baru,/jl. orde baru,,jl. orde baru,jl.\tB-STREET\norde\tI-STREET\nbaru\tI-STREET\...
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede,toko bb kids,raya samb gede,raya\tB-STREET\nsamb\tI-STREET\ngede\tI-STREET...
...,...,...,...,...,...
265883,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya,,kakap raya,karawaci\tO\nbaru\tO\nkakap\tB-STREET\nraya\tI...
265884,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani,jend\tB-STREET\nahmad\tI-STREET\nyani\tI-STREE...
265885,"raya cila kko , cilandak timur kel.",/raya cila kko,,raya cila kko,raya\tB-STREET\ncila\tI-STREET\nkko\tI-STREET\...
265886,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,,jalan\tO\ncipadu\tO\njaya\tO\ntaman\tB-POI\nas...


In [47]:
final = df.sample(frac=1, random_state=0).reset_index(drop=True)
final

,raw_address,POI/street,POI,Street,Label
0,"g obos xii , cyber computer pky , menteng jeka...",cyber computer pky/g obos xii,cyber computer pky,g obos xii,"g\tB-STREET\nobos\tI-STREET\nxii\tI-STREET\n,\..."
1,"siswa angk 45 , wonosobo timur wonosobo",siswa/angk 45,siswa,angk 45,"siswa\tB-POI\nangk\tB-STREET\n45\tI-STREET\n,\..."
2,"villa tomang baru , no 16c pasar kemis",/villa tomang baru,,villa tomang baru,villa\tB-STREET\ntomang\tI-STREET\nbaru\tI-STR...
3,"humanika solutama consulting , dr mansyur , pa...",humanika solutama consulting/dr mansyur,humanika solutama consulting,dr mansyur,humanika\tB-POI\nsolutama\tI-POI\nconsulting\t...
4,sudimara barat griya kenc 1 rt 3 14 15151 ciledug,/griya kenc,,griya kenc,sudimara\tO\nbarat\tO\ngriya\tB-STREET\nkenc\t...
...,...,...,...,...,...
265883,purn raya no 64 tanjung karang barat,/purn raya,,purn raya,purn\tB-STREET\nraya\tI-STREET\nno\tO\n64\tO\n...
265884,jl raya tange 14 tangerang,/jl raya tange,,jl raya tange,jl\tB-STREET\nraya\tI-STREET\ntange\tI-STREET\...
265885,gent dura genteng genteng,/gent dura,,gent dura,gent\tB-STREET\ndura\tI-STREET\ngenteng\tO\nge...
265886,"peru i , durian sei sungai ambawang",/peru i,,peru i,"peru\tB-STREET\ni\tI-STREET\n,\tO\ndurian\tO\n..."


In [48]:
n = len(final)
nt = int(0.95 * n)
train = final[:nt+1]
test = final[nt:]

In [49]:
train

,raw_address,POI/street,POI,Street,Label
0,"g obos xii , cyber computer pky , menteng jeka...",cyber computer pky/g obos xii,cyber computer pky,g obos xii,"g\tB-STREET\nobos\tI-STREET\nxii\tI-STREET\n,\..."
1,"siswa angk 45 , wonosobo timur wonosobo",siswa/angk 45,siswa,angk 45,"siswa\tB-POI\nangk\tB-STREET\n45\tI-STREET\n,\..."
2,"villa tomang baru , no 16c pasar kemis",/villa tomang baru,,villa tomang baru,villa\tB-STREET\ntomang\tI-STREET\nbaru\tI-STR...
3,"humanika solutama consulting , dr mansyur , pa...",humanika solutama consulting/dr mansyur,humanika solutama consulting,dr mansyur,humanika\tB-POI\nsolutama\tI-POI\nconsulting\t...
4,sudimara barat griya kenc 1 rt 3 14 15151 ciledug,/griya kenc,,griya kenc,sudimara\tO\nbarat\tO\ngriya\tB-STREET\nkenc\t...
...,...,...,...,...,...
252589,"sma negeri 4 banjarbaru ,",sma negeri 4 banjarbaru/,sma negeri 4 banjarbaru,,sma\tB-POI\nnegeri\tI-POI\n4\tI-POI\nbanjarbar...
252590,"tk pgri puspa indah , pringgarata",tk pgri puspa indah/,tk pgri puspa indah,,tk\tB-POI\npgri\tI-POI\npuspa\tI-POI\nindah\tI...
252591,ruko gate sawotratap gedangan,/ruko gate,,ruko gate,ruko\tB-STREET\ngate\tI-STREET\nsawotratap\tO\...
252592,"holland bakery , spbu kalianda , jl. lintas su...",spbu kalianda/jl. lintas sumatera,spbu kalianda,jl. lintas sumatera,"holland\tO\nbakery\tO\n,\tO\nspbu\tB-POI\nkali..."


In [50]:
def save_file(filename, arr) :
    f = open(filename, "w+",  encoding='utf-8')
    for s in arr :
        f.write(s.encode("ascii", "ignore").decode())
        f.write("\n")
    f.close()

In [51]:
train_path = data_path + "bert-fine-tune/train.txt"
validation_path = data_path + "bert-fine-tune/validation.txt"

train_data = train["Label"].values.tolist()
test_data = test["Label"].values.tolist()


ntry = None
# ntry = 1000
if ntry :
    train_data = train_data[:ntry]
    test_data = test_data[:ntry]

save_file(train_path, train_data)
save_file(validation_path, test_data)